In [48]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import dataset

from torch import Tensor, nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer



In [49]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


device = try_gpu(0)

In [50]:
import regex as re

with open('../data/simple.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'\s*[\u0964]\s*', r'\u0020\u0964\u0020', text)
    text = re.sub(r'\s*[\u003f]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*[\u002c]\s*', r'\u0020\u003f\u0020', text)
    text = re.sub(r'\s*\n\s*','\n', text)
    #text = re.sub(r'\s*[\u0966-\u0976]+\s*','\u0020[\u0966-\u0976]\u0020', text)
    #text = re.sub(r'\s+?\s+', r'\u0020?\u0020', text)
    text = re.sub(r'[^\u0900-\u097F,?\s+]','', text)
    
    

In [51]:
c = re.sub(r'[^\u0900-\u097F,?]','','?नमस्तेnishant,')
#c = re.sub(r'\s*[\u002c]\s*','\u0020\u002c\u0020','?नमस्तेnishant,')
c


'?नमस्ते,'

In [52]:
hex(ord('०'))

'0x966'

In [53]:
train_iter = text.split('\n')[:1200]
test_iter = text.split('\n')[1200:]

In [54]:
train_iter

['त्ररूपरिच्छेद',
 'आठौं योजनाको समीक्षा',
 '१ पृष्ठभूमि',
 'नेपालमा योजनावद्ध आर्थिक विकासको थालनी विसं २०१३ देखि पहिलो पंचवर्षीय योजना २०१३१८ को कार्यान्वयनबाट भएको हो । हालसम्म आठवटा आवधिक योजना कार्यान्वयनमा आइसकेकाछन् । विगत चार दशकभन्दा बढीको योजनावद्ध विकासबाट देश विकासलाई सघाउ पुर्याउने खालकासामाजिक तथा आर्थिक पूर्वाधारहरुको सिर्जनामा केही प्रगति हासिल हुन सकेको भए तापनि अपेक्षागरेजति उपलब्धि भने प्राप्त हुन सकेको छैन । जनताको जीवनयापनमा प्रमुख योगदान गर्ने कृषि',
 'क्षेत्रमा खासै प्रगति नभएको ? सीमित मात्रामा उद्योगहरु स्थापना हुन सकेको ? आन्तरिक बचतपरिचालन न्यून रहेको ? विदेशी सहायताको उपयोगमा खासै प्रगति हुन नसकेको ? वैदेशिक व्यापार घाटामा वृद्धि हुँदै गएको र बेरोजगारी तथा आर्थिक असमानता कम हुन नसकेको कारण गरिबीकोसमस्यामा सुधार आउन सकेन । जनसंख्या वृद्धिदरमा खासै कमी नआउनु र जनताको आयस्तरमा न्यून वृद्धि हुनुजस्ता कारणले गरिबीएक प्रमुख समस्याको रुपमा अझै विद्यमान छ । कुल जनसंख्याको ४२ प्रतिशत जनता गरिबीको रेखामुनि रहेको हालको परिप्रेक्ष्यमा एक निश्चित अवधिभित्र गरिबीको रेखामु

1200 396


103428608

In [108]:
tokenizer = get_tokenizer(None)
vocab = build_vocab_from_iterator(
    map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [109]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long)
            for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [110]:
train_data = data_process(train_iter)
test_data = data_process(test_iter)

In [111]:
train_data

tensor([11058,    51,    43,  ...,   445,   207,   874])

In [112]:
print(len(vocab))

17652


In [113]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.
    Args:
        data: Tensor, shape [N]
        bsz: int, batch size
    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [114]:
# batch_size = 35
# eval_batch_size = 10

# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# test_data = batchify(test_data, eval_batch_size)

In [115]:
text = ['आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य']
#text = ['जनसंख्या']
sample_data = data_process(
    text)

In [116]:
sample_data

tensor([ 8336,     0,   470,     0,   267,     0,     0,   516,  3422,  1599,
        15698,   591,     0,   516,  3422])

In [117]:
sample_data = batchify(sample_data, 2)
print("Given word:", text[0])
sample_data

Given word: आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य


tensor([[ 8336,   516],
        [    0,  3422],
        [  470,  1599],
        [    0, 15698],
        [  267,   591],
        [    0,     0],
        [    0,   516]], device='cuda:0')

In [118]:
bptt = 35
train_data = batchify(train_data, bptt)  # shape [seq_len, batch_size]
test_data = batchify(test_data, bptt)


import math
def get_batch(source: Tensor, i: int) -> tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int
    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    #target = source[i+1:i+1+seq_len]
    return data, target

In [119]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)
                             * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)





class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(
            d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]
        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [120]:
ntokens = len(vocab)  # size of vocabulary
emsize = 300  # embedding dimension
d_hid = 400  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # number of heads in nn.MultiheadAttention
dropout = 0.05  # dropout probability
model = TransformerModel(ntokens, emsize,nhead, d_hid,
                         nlayers, dropout).to(device)

In [121]:
criterion = nn.CrossEntropyLoss()
lr = 2  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [122]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [123]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        #print(type(output))
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [124]:
softmax = nn.Softmax(dim=2)
softmax = nn.LogSoftmax(dim=2)

In [125]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_softmax = softmax(output)
            output_softmax_permuted = output_softmax.permute(1, 0, 2)
            indices = torch.argmax(output_softmax_permuted, dim=2)
            target_indices = targets.t()
            # print(output)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()

    return total_loss / (len(eval_data) - 1)


In [126]:
import time
def generator(model: nn.Module, gen_data: Tensor, no_words = 10):
    model.eval()
    temp_text = text
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    pred_text = []
    for i in range(no_words):
        print('i:', i)
        batch_size = gen_data.size(0)
        if batch_size != bptt:
            src_mask_ = src_mask[:batch_size, :batch_size]
        output_softmax = model(gen_data, src_mask_)
        output_softmax_permuted = output_softmax.permute(1, 0, 2)
        indices = torch.argmax(output_softmax_permuted, dim=2)
        #print(indices[0],indices[1])
        #for j in range(batch_size):
        print('next word: ', [vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        print(i,"Gen_data: ",gen_data,"Pred_data: ",indices)
        pred_text.append([vocab.lookup_tokens(list(index))
                                  for index in indices][0][-1])
        if(batch_size <= 10):
            gen_data = torch.cat((gen_data[:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
        else:
            gen_data = torch.cat((gen_data[1:,:],indices.t()[-1:][:]),0)
            batch_size= gen_data.size(0)
            
    return pred_text



In [127]:
#            print('data')
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in data.t()]))
#             print(indices)
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in indices]))
#             print(len(targets))
#             print(list([vocab.lookup_tokens(list(index))
#                         for index in target_indices]))

In [128]:
        # target_indices = targets.t()
        # print(indices.shape)
        # print(indices)
        # temp_gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0][i]
        # gen_data = [vocab.lookup_tokens(
        #    list(index)) for index in indices][0]
        # print(temp_text)
        # print([[vocab.lookup_tokens(list(index))
        #        for index in indices][0][i]])
#         temp_text = [[vocab.lookup_tokens(list(index))]
#                       for index in indices[0][i]]
        # print(temp_text)
#         temp_text = [' '.join(temp_text)]
        # print(temp_text)
        # gen_data = vocab.lookup_tokens((list(gen_data))) + list(temp_gen_data)
        # gen_data = torch.tensor(
        #    (list(gen_data[0])+list(indices[0][i]))).unsqueeze(0)
        # gen_data = torch.concat([gen_data, torch.tensor(indices[0][i])], dim=1)
        #gen_data = ' '.join(gen_data)
        #return temp_txt
#         gen_data = data_process(temp_text)
#         gen_data = batchify(gen_data, 1)
        #return gen_data

In [129]:
# Loop over epochs. Save the model if the validation loss is the best
# we've seen so far. Adjust the learning rate after each epoch.
import copy
best_val_loss = float('inf')
epochs = 10
best_model = None
print(len(vocab.get_stoi()))
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    eval_loss = evaluate(model, test_data)
    eval_ppl = math.exp(eval_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
         f'valid loss {eval_loss:5.2f} | valid ppl {eval_ppl:8.2f}')
    print('-' * 89)
    #best_model = copy.deepcopy(model)
    if eval_loss < best_val_loss:
       best_val_loss = eval_loss
       best_model = copy.deepcopy(model)

#     scheduler.step()
#save model
torch.save(best_model.state_dict(),'best_model_3prex1005.pt')


17652
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  6.44s | valid loss  7.39 | valid ppl  1625.87
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  5.07s | valid loss  7.17 | valid ppl  1303.38
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  5.07s | valid loss  7.02 | valid ppl  1123.92
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  5.06s | valid loss  7.01 | valid ppl  1107.96
--------------------------------------------------------------------

KeyboardInterrupt: 

In [78]:
#Load saved model
model.load_state_dict(torch.load('best_model_3prex1005.pt'))
model.to(device)
best_model = model

In [79]:

print(sample_data[:,-1].unsqueeze(1))
print(sample_data)
z = generator(best_model, sample_data[:,-1].unsqueeze(1),no_words = 50)

tensor([[  516],
        [ 3422],
        [ 1599],
        [15698],
        [  591],
        [    0],
        [  516]], device='cuda:0')
tensor([[ 8336,   516],
        [    0,  3422],
        [  470,  1599],
        [    0, 15698],
        [  267,   591],
        [    0,     0],
        [    0,   516]], device='cuda:0')
i: 0
next word:  लगानीमा
0 Gen_data:  tensor([[  516],
        [ 3422],
        [ 1599],
        [15698],
        [  591],
        [    0],
        [  516]], device='cuda:0') Pred_data:  tensor([[ 370,   62, 1979,    2,   74,    2,  922]], device='cuda:0')
i: 1
next word:  वृद्धि
1 Gen_data:  tensor([[  516],
        [ 3422],
        [ 1599],
        [15698],
        [  591],
        [    0],
        [  516],
        [  922]], device='cuda:0') Pred_data:  tensor([[ 370,   62, 1979,    2,   74,    2,  922,   29]], device='cuda:0')
i: 2
next word:  भएको
2 Gen_data:  tensor([[  516],
        [ 3422],
        [ 1599],
        [15698],
        [  591],
        [    0],
    

i: 35
next word:  योजनामा
35 Gen_data:  tensor([[281],
        [283],
        [ 20],
        [123],
        [ 47],
        [  5],
        [  1],
        [ 51],
        [ 43],
        [334],
        [ 48]], device='cuda:0') Pred_data:  tensor([[248,  20, 123,  47,   5,   1,  51,  43, 334,  48,  90]],
       device='cuda:0')
i: 36
next word:  कुल
36 Gen_data:  tensor([[283],
        [ 20],
        [123],
        [ 47],
        [  5],
        [  1],
        [ 51],
        [ 43],
        [334],
        [ 48],
        [ 90]], device='cuda:0') Pred_data:  tensor([[ 20, 123,  47,   5,   1,  51,  43, 334,  48,  90,  46]],
       device='cuda:0')
i: 37
next word:  गार्हस्थ्य
37 Gen_data:  tensor([[ 20],
        [123],
        [ 47],
        [  5],
        [  1],
        [ 51],
        [ 43],
        [334],
        [ 48],
        [ 90],
        [ 46]], device='cuda:0') Pred_data:  tensor([[ 65,  47,   5,   1,  51,  90, 334,  48,  90,  46, 281]],
       device='cuda:0')
i: 38
next word:  बचत
38 G

In [81]:
'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य ' +' '.join(z)

'संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त राज्य लगानीमा वृद्धि भएको छ । आठौं योजनाको तुलनामा नवौं योजनामा कुल गार्हस्थ्य बचत हुने अनुमान गरिएको छ । आठौं योजनाको तुलनामा नवौं योजनामा कुल गार्हस्थ्य बचत हुने अनुमान गरिएको छ । आठौं योजनाको तुलनामा नवौं योजनामा कुल गार्हस्थ्य बचत हुने अनुमान गरिएको छ । आठौं योजनाको तुलनामा नवौं योजनामा कुल'

In [82]:
evaluate(best_model, sample_data)

9.959761619567871

In [83]:
'''
आधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , 
संयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त

विकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय
'''

'\nआधिकारिक निर्णयको कारणले , वाणिज्य बिभागले , \nसंयुक्त राज्य अमेरिकी समुद्री पानी निर्माताद्वारा संयुक्त\n\nविकास गर्न प्रोत्साहित गरी सहकारी क्षेत्रले आर्थिक दृष्टिले सक्रिय\n'

In [100]:
st = ['गर्ने']
st_i = data_process(st)

In [101]:
st_i = st_i.unsqueeze(1).to(device)

In [104]:
z_ = generator(best_model, st_i,no_words =40 )

i: 0
next word:  व्यवस्था
0 Gen_data:  tensor([[9]], device='cuda:0') Pred_data:  tensor([[21]], device='cuda:0')
i: 1
next word:  मिलाइनेछ
1 Gen_data:  tensor([[ 9],
        [21]], device='cuda:0') Pred_data:  tensor([[ 21, 245]], device='cuda:0')
i: 2
next word:  ।
2 Gen_data:  tensor([[  9],
        [ 21],
        [245]], device='cuda:0') Pred_data:  tensor([[ 21, 245,   1]], device='cuda:0')
i: 3
next word:  ङ्ग
3 Gen_data:  tensor([[  9],
        [ 21],
        [245],
        [  1]], device='cuda:0') Pred_data:  tensor([[ 21, 245,   1,  10]], device='cuda:0')
i: 4
next word:  विशेष
4 Gen_data:  tensor([[  9],
        [ 21],
        [245],
        [  1],
        [ 10]], device='cuda:0') Pred_data:  tensor([[ 21, 245,   1,  10,  99]], device='cuda:0')
i: 5
next word:  क्षेत्रमा
5 Gen_data:  tensor([[  9],
        [ 21],
        [245],
        [  1],
        [ 10],
        [ 99]], device='cuda:0') Pred_data:  tensor([[ 21, 245,   1,  10,  99,  17]], device='cuda:0')
i: 6
next word:  

In [105]:
' '.join(z_)

'व्यवस्था मिलाइनेछ । ङ्ग विशेष क्षेत्रमा सञ्चालन गरी उपयुक्त प्रविधि विकास गरी उत्पादन र उत्पादकत्व वृद्धि गरी उत्पादकत्व वृद्धि गरिनेछ । ङ्ग कृषि उत्पादन तथा उत्पादकत्व वृद्धि ? प्रशोधन र उत्पादकत्व वृद्धि जस्ता कार्यक्रमहरु सञ्चालन गरी सञ्चालन गरिने छन् ।'